<a href="https://colab.research.google.com/github/tahamsi/tahamsi/blob/main/Topic_Modeling_with_BerTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://medium.com/grabngoinfo/topic-modeling-with-deep-learning-using-python-bertopic-cf91f5676504
#https://medium.com/grabngoinfo/hyperparameter-tuning-for-bertopic-model-in-python-104445778347
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
# Data processing
import pandas as pd
import numpy as np
# Text preprocessiong
import nltk
nltk.download('stopwords')
nltk.download('omw-1.4')
nltk.download('wordnet')
wn = nltk.WordNetLemmatizer()
# Topic model
from bertopic import BERTopic
# Dimension reduction
from umap import UMAP
# Clustering
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
# Count vectorization
from sklearn.feature_extraction.text import CountVectorizer
# Sentence transformer
from sentence_transformers import SentenceTransformer
# Flair
from transformers.pipelines import pipeline
from flair.embeddings import TransformerDocumentEmbeddings, WordEmbeddings, DocumentPoolEmbeddings, StackedEmbeddings

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from io import BytesIO
from zipfile import ZipFile
import urllib.request
    
url = urllib.request.urlopen("https://github.com/Ali-Alameer/NLP/raw/main/data/NIPS%20Papers.zip")

with ZipFile(BytesIO(url.read())) as my_zip_file:
    temp = my_zip_file.open('NIPS Papers/papers.csv')

pr_cancer_papers = pd.read_csv(temp)
# to minimise compute 
pr_cancer_papers = pr_cancer_papers.iloc[0:1000]
# Print head
pr_cancer_papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [ ]:
# Get the dataset information
pr_cancer_papers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   year        1000 non-null   int64 
 2   title       1000 non-null   object
 3   event_type  0 non-null      object
 4   pdf_name    1000 non-null   object
 5   abstract    1000 non-null   object
 6   paper_text  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [ ]:
# Remove stopwords
stopwords = nltk.corpus.stopwords.words('english')
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 179 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [ ]:
# Remove stopwords
pr_cancer_papers['abstract_without_stopwords'] = pr_cancer_papers['paper_text'].apply(lambda x: ' '.join([w for w in x.split() if w.lower() not in stopwords]))
# Lemmatization
pr_cancer_papers['abstract_lemmatized'] = pr_cancer_papers['abstract_without_stopwords'].apply(lambda x: ' '.join([wn.lemmatize(w) for w in x.split() if w not in stopwords]))
# Take a look at the data
pr_cancer_papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text,abstract_without_stopwords,abstract_lemmatized
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,767 SELF-ORGANIZATION ASSOCIATIVE DATABASE APP...,767 SELF-ORGANIZATION ASSOCIATIVE DATABASE APP...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,683 MEAN FIELD THEORY LAYER IV VISUAL CORTEX A...,683 MEAN FIELD THEORY LAYER IV VISUAL CORTEX A...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,394 STORING COVARIANCE ASSOCIATIVE LONG?TERM P...,394 STORING COVARIANCE ASSOCIATIVE LONG?TERM P...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...,Bayesian Query Construction Neural Network Mod...,Bayesian Query Construction Neural Network Mod...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a...","Neural Network Ensembles, Cross Validation, Ac...","Neural Network Ensembles, Cross Validation, Ac..."


In [ ]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Clustering model
# hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples = 5, 
# metric='euclidean', prediction_data=True)
kmeans_model = KMeans(n_clusters=9)
# Initiate a sentence transformer model
sentence_model = SentenceTransformer("paraphrase-albert-small-v2")
# Initiate a pretrained model
hf_model = pipeline("feature-extraction", model="distilroberta-base")

# Initiate a pretrained embedding model
roberta_model = TransformerDocumentEmbeddings('roberta-base')
# Initiate another pretrained embedding model
glove_embedding = WordEmbeddings('crawl')
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])
# Stack the two pretrained embedding models
stacked_embeddings = StackedEmbeddings(embeddings=[roberta_model, 
document_glove_embeddings])

# Count vectorizer
vectorizer_model = CountVectorizer(min_df=10)

# Initiate BERTopic
# topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True,hdbscan_model=kmeans_model,
#                        embedding_model=stacked_embeddings,min_topic_size=5, n_gram_range=(1, 3),diversity=0.8)#vectorizer_model=vectorizer_model)# Other options for embedding_model are sentence_model, hf_model,roberta_model

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, language="english", calculate_probabilities=True,hdbscan_model=kmeans_model, n_gram_range=(1, 3))
# Run BERTopic model
topics, probabilities = topic_model.fit_transform(pr_cancer_papers['abstract_lemmatized'])#abstract_lemmatized

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


2023-03-01 13:15:53,565 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M.vectors.npy not found in cache, downloading to /tmp/tmp6ykh3rih


100%|██████████| 1200000128/1200000128 [00:48<00:00, 24580257.24B/s]

2023-03-01 13:16:42,756 copying /tmp/tmp6ykh3rih to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M.vectors.npy


2023-03-01 13:16:47,227 removing temp file /tmp/tmp6ykh3rih
2023-03-01 13:16:47,885 https://flair.informatik.hu-berlin.de/resources/embeddings/token/en-fasttext-crawl-300d-1M not found in cache, downloading to /tmp/tmpfi43ld8v


100%|██████████| 39323680/39323680 [00:02<00:00, 17780217.61B/s]

2023-03-01 13:16:50,469 copying /tmp/tmpfi43ld8v to cache at /root/.flair/embeddings/en-fasttext-crawl-300d-1M
2023-03-01 13:16:50,501 removing temp file /tmp/tmpfi43ld8v


In [ ]:
# Get the list of topics
topic_model.get_topic_info()

,Topic,Count,Name
0,0,164,0_model_data_algorithm_distribution
1,1,129,1_network_learning_function_training
2,2,126,2_model_visual_cell_figure
3,3,121,3_neuron_input_network_spike
4,4,118,4_set_algorithm_training_function
5,5,108,5_image_model_recognition_set
6,6,99,6_state_learning_policy_value
7,7,72,7_network_model_speech_state
8,8,63,8_signal_source_frequency_sound


In [ ]:
# Get top 10 terms for a topic
topic_model.get_topic(0)

[('model', 0.013567008799832777),
 ('data', 0.012840824292542742),
 ('algorithm', 0.009449539224843378),
 ('distribution', 0.009123155907851113),
 ('gaussian', 0.008158799686569301),
 ('network', 0.007228280268411163),
 ('function', 0.0070727501777283625),
 ('set', 0.006955337542311291),
 ('bayesian', 0.006629657685026249),
 ('parameter', 0.006625129471435716)]

In [ ]:
# Visualize top topic keywords
topic_model.visualize_barchart(top_n_topics=12)

In [ ]:
# Visualize term rank decrease
topic_model.visualize_term_rank()

In [ ]:
# Visualize intertopic distance
topic_model.visualize_topics()

In [ ]:
# Visualize connections between topics using hierachical clustering
topic_model.visualize_hierarchy(top_n_topics=10)

In [ ]:
# Visualize similarity using heatmap
topic_model.visualize_heatmap()

In [ ]:
# Get the topic predictions
topic_prediction = topic_model.topics_[:]
# Save the predictions in the dataframe
pr_cancer_papers['topic_prediction'] = topic_prediction
# Take a look at the data
pr_cancer_papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text,abstract_without_stopwords,abstract_lemmatized,topic_prediction
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...,767 SELF-ORGANIZATION ASSOCIATIVE DATABASE APP...,767 SELF-ORGANIZATION ASSOCIATIVE DATABASE APP...,5
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...,683 MEAN FIELD THEORY LAYER IV VISUAL CORTEX A...,683 MEAN FIELD THEORY LAYER IV VISUAL CORTEX A...,2
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...,394 STORING COVARIANCE ASSOCIATIVE LONG?TERM P...,394 STORING COVARIANCE ASSOCIATIVE LONG?TERM P...,3
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...,Bayesian Query Construction Neural Network Mod...,Bayesian Query Construction Neural Network Mod...,0
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a...","Neural Network Ensembles, Cross Validation, Ac...","Neural Network Ensembles, Cross Validation, Ac...",4


In [ ]:
'''
# New data for the review
new_review = "I like the new headphone. Its sound quality is great."
# Find topics
num_of_topics = 3
similar_topics, similarity = topic_model.find_topics(new_review, top_n=num_of_topics); 
# Print results
print(f'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}')
'''

'\n# New data for the review\nnew_review = "I like the new headphone. Its sound quality is great."\n# Find topics\nnum_of_topics = 3\nsimilar_topics, similarity = topic_model.find_topics(new_review, top_n=num_of_topics); \n# Print results\nprint(f\'The top {num_of_topics} similar topics are {similar_topics}, and the similarities are {np.round(similarity,2)}\')\n'

In [ ]:
'''
# Print the top keywords for the top similar topics
for i in range(num_of_topics):
  print(f'The top keywords for topic {similar_topics[i]} are:')
  print(topic_model.get_topic(similar_topics[i]))
'''

"\n# Print the top keywords for the top similar topics\nfor i in range(num_of_topics):\n  print(f'The top keywords for topic {similar_topics[i]} are:')\n  print(topic_model.get_topic(similar_topics[i]))\n"

In [ ]:
# Save the topic model
topic_model.save("pr_cancer_papers_topic_model")	
# Load the topic model
my_model = BERTopic.load("pr_cancer_papers_topic_model")